In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('analyzed_application_data.csv')

In [3]:
dataset

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,1,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,0,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,1,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,0,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,100007,0,0,1,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,1,0,0,0,157500.0,254700.0,27558.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
307507,456252,0,0,0,0,1,0,72000.0,269550.0,12001.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
307508,456253,0,0,0,0,1,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,0,0,0,1,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
dataset.columns

Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)

In [6]:
# Drop multiple columns 'TARGET' and 'SK_ID_CURR'
indep = dataset.drop(['TARGET', 'SK_ID_CURR'], axis=1)

# Select the dependent variable 'TARGET'
dep = dataset['TARGET']


In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 



grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 



Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [10]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [11]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'lbfgs'}: 0.8830841272286888


In [12]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[94207   102]
 [ 8117    78]]


In [15]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     94309
           1       0.43      0.01      0.02      8195

    accuracy                           0.92    102504
   macro avg       0.68      0.50      0.49    102504
weighted avg       0.88      0.92      0.88    102504



In [16]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


0.7449970150243655

In [17]:
table=pd.DataFrame.from_dict(re)

In [18]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,92.260483,42.345425,1.062068,1.305581,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.881512,0.880884,0.881358,0.881829,0.881842,0.881485,0.000353,3
1,7.489673,0.253778,0.107228,0.003868,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.881441,0.880942,0.881429,0.881933,0.881746,0.881498,0.000337,1
2,140.436951,13.404729,0.230259,0.266253,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.881512,0.880884,0.881358,0.881829,0.881868,0.881490,0.000359,2
3,145.442856,14.949260,0.118030,0.029901,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.881185,0.880959,0.881316,0.881580,0.881656,0.881339,0.000256,4


In [21]:

# Prepare the input data in the correct format
input_data = np.array([[1, 1, 1, 0, 0, 202500, 406597.5, 24700.5, 351000, 0, 1, 1, 1, 0.01880, 1, -9461, -637, -3648, -2120, 1, 1, 0, 1, 1, 0, 1, 1, 2, 2, 1, 10, 0, 0, 0, 0, 0, 0, 1, 3, 0.083036967, 0.262948593, 0.13937578, 0.0247, 0.0369, 0.9722, 0.6192, 0.0143, 0, 0.069, 0.0833, 0.125, 0.0369, 0.0202, 0.019, 0, 0, 0.0252, 0.0383, 0.9722, 0.6341, 0.0144, 0, 0.069, 0.0833, 0.125, 0.0377, 0.022, 0.0198, 0, 0, 0.025, 0.0369, 0.9722, 0.6243, 0.0144, 0, 0.069, 0.0833, 0.125, 0.0375, 0.0205, 0.0193, 0, 0, 1, 1, 0.0149, 1, 0, 2, 2, 2, 2, -1134, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

# Make predictions
Future_Prediction = grid.predict(input_data)
print("Future_Prediction={}".format(Future_Prediction))


Future_Prediction=[1]


In [22]:
import pickle

In [23]:
filename = "FRD_Finalized_Losistic_Model.sav"
pickle.dump(grid, open(filename, 'wb'))